# Парсинг таблицы с сайта на Python:
### Пошаговое руководство

In [ ]:
# https://idatica.com/blog/parsing-tablitsy-s-sayta-na-python-poshagovoye-rukovodstvo/

## ШАГ 1. УСТАНОВКА БИБЛИОТЕК

Прежде всего, нужно установить в нашу среду разработки библиотеки:

BeautifulSoup4
Requests
pandas
lxml

## ШАГ 2. ИМПОРТИРОВАНИЕ БИБЛИОТЕК

In [ ]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

## ШАГ 3. ВЫБОР СТРАНИЦЫ

В этом проекте мы будем парсить таблицу данных covid с Worldometers.
Как и в предыдущем руководстве, данный веб-сайт также создан с помощью HTML и считается более легким для понимания новичками.

https://www.worldometers.info/coronavirus/

## ШАГ 4. ЗАПРОС РАЗРЕШЕНИЯ

После выбора страницы для парсинга мы можем скопировать ее URL-адрес и использовать requests, чтобы запросить разрешение у сервера на получение данных с их сайта.

In [ ]:
# Create an URL object
url = 'https://www.worldometers.info/coronavirus/'
# Create object page
page = requests.get(url)

Результат <Response [200]> означает, что сервер позволяет нам собирать информацию.
Дальше необходимо обработать HTML-код с помощью lxml, чтобы сделать его более доступным для чтения.

In [ ]:
# parser-lxml = Change html to Python friendly format
# Obtain page's information
soup = BeautifulSoup(page.text, 'lxml')
soup

## ШАГ 5. ПРОСМОТР КОДА ЭЛЕМЕНТОВ ТАБЛИЦЫ

В предыдущей статье мы узнали, как просмотреть код каждого элемента на странице веб-сайта.
Чтобы получить информацию о коде элементов таблицы, нам нужно сначала проверить ее расположение.

https://idatica.com/blog/parsing-saytov-na-python-rukovodstvo-dlya-novichkov/

Как видно из рисунка выше, эта таблица находится внутри тега <table> и id = ‘main_table_countries_today’.
Теперь мы можем определить переменную. В нашем случае мы определим таблицу как ‘table1’.

In [ ]:
# Obtain information from tag <table>
table1 = soup.find(‘table’, id=’main_table_countries_today’)
table1

## ШАГ 6. СОЗДАНИЕ КОЛОНОК

После создания table1 мы можем посмотреть расположение каждой колонки.
Если мы просмотрим все колонки, то заметим, что они имеют одну и ту же характеристику.

На рисунке видна общая характеристика каждой колонки – они расположены внутри тега <th>.

In [ ]:
# После нахождения тегов мы создаем цикл for, чтобы заполнить пустой список нашими колонками.
# Давайте определим пустой список как headers.

# Obtain every title of columns with tag <th>
headers = []
for i in table1.find_all(‘th’):
title = i.text
headers.append(title)

# Результат:

Список успешно заполнен и мы можем проверить его еще раз.
Давайте посмотрим на индекс 13, здесь многострочный текст.
Подобный перенос может быть проблемой, когда мы захотим сделать из него фрейм данных, поэтому необходимо преобразовать его в однострочный текст.

In [ ]:
# Convert wrapped text in column 13 into one line text
headers[13] = ‘Tests/1M pop’

# Результат:

## ШАГ 7. СОЗДАНИЕ ФРЕЙМА ДАННЫХ

In [ ]:
# Следующий этап парсинга таблицы с помощью Python — это создание фрейма данных.
# Предлагаю определить фрейм данных как mydata.

# Create a dataframe
mydata = pd.DataFrame(columns = headers)

## ШАГ 8. СОЗДАНИЕ ЦИКЛА FOR ДЛЯ ЗАПОЛНЕНИЯ ФРЕЙМА ДАННЫХ

Как только фрейм данных готов, мы можем заполнить его элементами в каждой колонке.
До момента создания цикла for нам нужно еще определить место расположения строки и колонки элемента.

На рисунках выше видно, что строка находится внутри тега <tr>, а элементы — внутри тега <td>.
Это применимо ко всем строкам и элементам в таблице.

In [ ]:
# Теги найдены и теперь можно создавать цикл for.

# Create a for loop to fill mydata
for j in table1.find_all(‘tr’)[1:]:
row_data = j.find_all(‘td’)
row = [i.text for i in row_data]
length = len(mydata)
mydata.loc[length] = row

# Результат:

## ШАГ 9. ОЧИСТКА ФРЕЙМА ДАННЫХ

Далее, как только фрейм данных будет успешно создан, мы можем удалить и очистить ненужные строки.
В нашем случае мы удалим индекс 0–6, 222–228, затем вернем индекс в исходное состояние и удалим столбец «#».

In [ ]:
# Drop and clearing unnecessary rows
mydata.drop(mydata.index[0:7], inplace=True)
mydata.drop(mydata.index[222:229], inplace=True)
mydata.reset_index(inplace=True, drop=True)

# Drop “#” column
mydata.drop(‘#’, inplace=True, axis=1)

# Результат:

## ШАГ 10. ЭКСПОРТ ДАННЫХ В CSV-ФАЙЛ

In [ ]:
# Когда фрейм данных будет готов, мы можем экспортировать данные в CSV-файл.

# Export to csv
mydata.to_csv(‘covid_data.csv’, index=False)

# Try to read csv
mydata2 = pd.read_csv(‘covid_data.csv’)

##### Это был завершающий этап парсинга таблицы на Python и теперь данные можно использовать для работы в проектах по анализу и обработке данных, в машинном обучении и для получения другой необходимой информации.